# IMPACT paper experiments
### 1. Init
#### 1.1. Import libraries (necessary)

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["PYTHONHASHSEED"] = "0"
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":4096:8"

from liriscat import utils
utils.set_seed(0)
from liriscat import dataset
from liriscat import selectionStrategy
from liriscat import CDM

import logging
import gc
import json
import torch
utils.set_seed(0)
import pandas as pd
from importlib import reload
import IMPACT

#### 1.2. Set up the loggers (recommended)

In [2]:
utils.setuplogger(verbose = True, log_name="liriscat")

### 2. CDM prediction
#### 2.1. Training and testing, sequential version

In [3]:
import warnings
import numpy as np

gc.collect()
torch.cuda.empty_cache()

reload(utils)
reload(selectionStrategy)
reload(CDM)
reload(dataset)

<module 'liriscat.dataset' from '/home/arthur/liriscat/liriscat/dataset/__init__.py'>

In [6]:
config = utils.generate_eval_config(load_params=True, esc = 'error', valid_metric= 'mi_acc', pred_metrics = ["mi_acc"], profile_metrics = ['doa'], save_params=False, n_query=4, num_epochs=4, batch_size=512)
utils.set_seed(config["seed"])

config["dataset_name"] = "math2"
logging.info(config["dataset_name"])
config['learning_rate'] = 0.02026
config['lambda'] = 1.2e-5
config['d_in'] = 4
config['num_responses'] = 12
#pred_metrics,df_interp = test(config)

CUDA is available. Using GPU.
[INFO 31:06] math2


In [7]:
logging.info(f'#### {config["dataset_name"]} ####')
logging.info(f'#### config : {config} ####')
config['embs_path']='../embs/'+str(config["dataset_name"])
config['params_path']='../ckpt/'+str(config["dataset_name"])

pred_metrics = {m:[] for m in config['pred_metrics']}
profile_metrics = {m:[] for m in config['profile_metrics']}

gc.collect()
torch.cuda.empty_cache()

# Dataset downloading for doa and rm
warnings.filterwarnings("ignore", message="invalid value encountered in divide")
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Concept map format : {question_id : [category_id1, category_id2, ...]}
concept_map = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_concept_map.json', 'r'))
concept_map = {int(k): [int(x) for x in v] for k, v in concept_map.items()}

## Metadata map format : {"num_user_id": ..., "num_item_id": ..., "num_dimension_id": ...}
metadata = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_metadata.json', 'r'))


## Tensor containing the nb of modalities per question
nb_modalities = torch.load(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_nb_modalities.pkl',weights_only=True)


[INFO 31:07] #### math2 ####
[INFO 31:07] #### config : {'seed': 0, 'dataset_name': 'math2', 'load_params': True, 'save_params': False, 'embs_path': '../embs/', 'params_path': '../ckpt/', 'early_stopping': True, 'esc': 'error', 'verbose_early_stopping': False, 'disable_tqdm': False, 'valid_metric': 'mi_acc', 'learning_rate': 0.02026, 'batch_size': 512, 'valid_batch_size': 10000, 'num_epochs': 4, 'eval_freq': 1, 'patience': 30, 'device': device(type='cuda'), 'lambda': 1.2e-05, 'tensorboard': False, 'flush_freq': True, 'pred_metrics': ['mi_acc'], 'profile_metrics': ['doa'], 'num_responses': 12, 'low_mem': False, 'n_query': 4, 'CDM': 'impact', 'i_fold': 0, 'num_inner_users_epochs': 10, 'num_inner_epochs': 10, 'inner_lr': 0.0001, 'inner_user_lr': 0.0001, 'd_in': 4} ####


In [8]:
i_fold = 0
## Dataframe columns : (user_id, question_id, response, category_id)
train_df = pd.read_csv(
    f'../datasets/2-preprocessed_data/{config["dataset_name"]}_train_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})
valid_df = pd.read_csv(
    f'../datasets/2-preprocessed_data/{config["dataset_name"]}_valid_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})
test_df = pd.read_csv(
    f'../datasets/2-preprocessed_data/{config["dataset_name"]}_test_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})

In [39]:
reload(dataset)
train_data = dataset.CATDataset(train_df, concept_map, metadata, config,nb_modalities)
valid_data = dataset.EvalDataset(valid_df, concept_map, metadata, config,nb_modalities)
test_data = dataset.EvalDataset(test_df, concept_map, metadata, config,nb_modalities)

In [40]:
config['inner_user_lr'] = 0.01
config['num_inner_users_epochs'] = 8

In [43]:
reload(utils)
reload(selectionStrategy)
reload(dataset)
reload(CDM)

# Important for reproducibility
train_data.reset_rng()
valid_data.reset_rng()
test_data.reset_rng()

S = selectionStrategy.Random(metadata,**config)
S.init_models(train_data, valid_data)
S.evaluate_test(test_data)

[INFO 14:09] Random_cont_model
compiling CDM model
compiling selection model


100%|██████████| 4/4 [00:08<00:00,  2.09s/it]


({0: {'mi_acc': 0.6769307255744934},
  1: {'mi_acc': 0.6893871426582336},
  2: {'mi_acc': 0.6989536285400391},
  3: {'mi_acc': 0.7060288786888123}},
 {0: {'doa': 0.5181272466001896},
  1: {'doa': 0.5157223498104185},
  2: {'doa': 0.5104920518505776},
  3: {'doa': 0.5120360390141414}})

In [23]:

S.train(train_data, valid_data)

[INFO 50:23] train on cuda
[INFO 50:23] -- START Training --


/home/arthurb/Programmation/liriscat/liriscat/selectionStrategy/random.py:39: UserWarning: get_params() Notimplemented
  warnings.warn('get_params() Notimplemented')
/home/arthurb/Programmation/liriscat/liriscat/dataset/dataset.py:188: UserWarning: The log_tensor contains both query and the meta set !
  warnings.warn("The log_tensor contains both query and the meta set !")
/home/arthurb/Programmation/liriscat/liriscat/dataset/dataset.py:193: UserWarning: The user_dict contains both query and the meta set !
  warnings.warn("The user_dict contains both query and the meta set !")
  0%|          | 0/4 [00:01<?, ?it/s]


KeyboardInterrupt: 